In [ ]:
import os
import cv2

def preprocess_images(input_dir, output_dir, img_size=(128, 128)):
    """
    Preprocess images by resizing, normalizing, and saving to an output directory.
    
    Args:
        input_dir (str): Path to the directory containing raw images.
        output_dir (str): Path to save preprocessed images.
        img_size (tuple): Desired image size (width, height).
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for class_name in os.listdir(input_dir):  # Iterate through class folders
        class_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_path):
            continue  # Skip non-folder files

        # Create class folder in output directory
        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                # Read the image
                img = cv2.imread(img_path)

                # Resize the image
                img_resized = cv2.resize(img, img_size)

                # Normalize pixel values to [0, 1]
                img_normalized = img_resized / 255.0

                # Optional: Apply denoising filter (if noisy images)
                img_denoised = cv2.fastNlMeansDenoisingColored(
                    (img_normalized * 255).astype("uint8"), None, 10, 10, 7, 21
                )

                # Save the preprocessed image
                output_path = os.path.join(output_class_path, img_name)
                cv2.imwrite(output_path, (img_denoised * 255).astype("uint8"))
                print(f"Processed and saved: {output_path}")
            except Exception as e:
                print(f"Error processing {img_path}: {e}")


# Define paths
input_directory = "F:/2025/Lofty Agrotech Internship/PADDY IMAGES/"  # Path to raw images
output_directory = "F:/2025/Lofty Agrotech Internship/processed_images/"  # Path to save preprocessed images

# Run preprocessing
preprocess_images(input_directory, output_directory)


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

In [ ]:
# Define paths
train_dir = 'F:/2025/Lofty Agrotech Internship/processed_images/'  # Replace with path to your training images
validation_dir = 'F:/2025/Lofty Agrotech Internship/processed_images/'  # Replace with path to your validation images
img_size = (128, 128)  # Image size to match model input
batch_size = 32

In [ ]:
# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'  # Multi-class classification
)

In [ ]:
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# Build CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 classes (9 diseases + normal)
])

In [ ]:
# Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

In [ ]:
# Save the Model
model.save('disease_classifier_model.h5')

In [ ]:
# Evaluate the Model on validation data
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {validation_accuracy:.2f}, Validation Loss: {validation_loss:.2f}")

In [ ]:
# Prediction function
def predict_disease(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Predict the class
    prediction = model.predict(img_array)

    
    # Get the predicted class index
    predicted_class_idx = np.argmax(prediction, axis=-1)
    
    # Class labels (9 diseases + normal)
    class_labels = ['bacterial_leaf_blight',
'bacterial_leaf_streak',
'bacterial_panicle_blight',
'blast',
'brown_spot',
'dead_heart',
'downy_mildew',
'hispa',
'normal',
'tungro']
    # Output the result
    print(f"Predicted class: {class_labels[predicted_class_idx[0]]}")

    # Display the image
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
# Example usage"C:\Users\Nandan\Desktop\5g"
img_path = 'C:/Users/Nandan/Desktop/intern/test_processed_images/diseased/blb.jpg'  # Replace with the actual test image path
predict_disease(img_path)

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()
